# 1. Подготовка данных

In [1]:
import warnings
warnings.filterwarnings('ignore')

import optuna
import mlflow
import joblib
import pandas as pd
import xgboost as xgb
import numpy as np
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, f1_score, balanced_accuracy_score

mlflow.set_tracking_uri("http://127.0.0.1:8000")
client=mlflow.MlflowClient(tracking_uri='http://127.0.0.1:8000')
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [2]:
df = pd.read_csv('Churn.csv')

In [3]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
# проверим наличие явных дубликатов:
df.duplicated().sum()

0

In [11]:
# посмотрим уникальные значения в категориальных признаках:
cat_columns = ['Geography', 'Tenure', 'NumOfProducts', 'Gender', 'HasCrCard', 'IsActiveMember']
for col in cat_columns:
    display(df[col].value_counts())

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

Tenure
1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: count, dtype: int64

NumOfProducts
1    5084
2    4590
3     266
4      60
Name: count, dtype: int64

Gender
Male      5457
Female    4543
Name: count, dtype: int64

HasCrCard
1    7055
0    2945
Name: count, dtype: int64

IsActiveMember
1    5151
0    4849
Name: count, dtype: int64

In [12]:
# посчитаем классы в целевом признаке:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

Можно отметить, что дубликаты в датасете отсутствуют, в столбце Tenure есть около тысячи пропусков, а также несколько столбцов лучше вообще удалить, из них:

- RowNumber - это индекс строки в данных, важной информации не содержит;
- CustomerId - уникальный идентификатор клиента, по нему мы проверили дубликаты и теперь его можно удалить;
- Surname - фамилия, которая тоже не имеет связи с оттоком клиентов.

После проверки датасета на сбалансированность классов в таргете мы видим, что объектов отрицательного класса намного больше (почти в четыре раза), но все равно уходит примерно 20% клиентов. В столбце Geography находятся всего 3 уникальных значения: France, Germany и Spain. В столбце Tenure (сколько лет человек является клиентом банка) есть 909 пропусков. Заполним их значением -1, добавив логический параметр add_indicator=True. Есть несколько бинарных признаков (HasCrCard, IsActiveMember), которывыделим в отдельную категорию признаков.

In [13]:
# удалим ненужные столбцы:
df.drop(["RowNumber", "CustomerId", "Surname"], axis=1, inplace=True)

In [14]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           9091 non-null   float64
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


In [16]:
# сохраним признаки и целевой признак в отдельных переменных:
y = df['Exited']
X = df.drop('Exited', axis=1)

In [17]:
# разобьем исходные данные на обучающую и тестовую выборки в соотношении 4:1:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345, stratify=y)

In [18]:
# посмотрим на размеры выборок:
print('Размер обучающей выборки:', X_train.shape)
print('Размер тестовой выборки:', X_test.shape)

Размер обучающей выборки: (8000, 10)
Размер тестовой выборки: (2000, 10)


In [19]:
# внесем в переменные списки все численные и категориальные признаки:
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
numeric_features = X_train.select_dtypes(exclude=['object']).columns.tolist()
binary_features = ['HasCrCard', 'IsActiveMember']
numeric_features = list(set(numeric_features) - set(binary_features))

In [20]:
numeric_features

['Balance', 'EstimatedSalary', 'CreditScore', 'NumOfProducts', 'Age', 'Tenure']

In [21]:
categorical_features

['Geography', 'Gender']

In [22]:
binary_features

['HasCrCard', 'IsActiveMember']

# 2. Обучение модели без учёта дисбаланса классов (XGBoost)

Для автоматизации процесса подготовим численный и категориальный пайплайны.
- Для категориальных признаков (Geography, Gender) используем One Hot Encoding.
- Для заполнения пропусков используем SimpleImputer.
- Выполним трансформацию Йео-Джонсона.
- Для моделей логистической регрессии и SVM выполним масштабирование данных; для моделей, основанных на деревьях, выполнять его не будем.
- Объединим пайплайны классом ColumnTransformer.

In [22]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer()) 
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)

], remainder='passthrough')

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', xgb.XGBClassifier(random_state=12345))
])

In [23]:
def optuna_optimize(trial):
    param_grid = {
        'model__n_estimators': trial.suggest_int('model__n_estimators', 50, 250),
        'model__max_depth': trial.suggest_int('model__max_depth', 2, 10),
        'model__eta': trial.suggest_float('model__eta', 0.01, 1.5)
    }

    pipe.set_params(**param_grid)
    score = cross_val_score(pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [24]:
study_xgb = optuna.create_study(study_name="XGBoost Optuna Optimization Imbalanced",
                                direction='maximize')

[I 2023-11-28 09:42:51,979] A new study created in memory with name: XGBoost Optuna Optimization Imbalanced


In [25]:
study_xgb.optimize(optuna_optimize, n_trials=100)

In [26]:
print("Наилучшие значения гиперпараметров XGBoost: {}".format(study_xgb.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_xgb.best_value))

Наилучшие значения гиперпараметров XGBoost: {'model__n_estimators': 102, 'model__max_depth': 3, 'model__eta': 0.2039844245783137}
f1_score на обучающем наборе: 0.590


In [27]:
with mlflow.start_run(nested=True,
                      run_name='xgb_pipeline_imbalanced'):

    best_params = study_xgb.best_params
    best_score = study_xgb.best_value
    mlflow.log_params(best_params)

    pipe.set_params(**best_params)
    pipe.fit(X_train, y_train)
    pred_proba = pipe.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba)),
        'f1': f1_score(y_test, np.round(pred_proba))
    })

    mlflow.sklearn.log_model(pipe, 'xgb_pipeline_imbalanced')

# 3. Обучение моделей с учётом дисбаланса классов
## 3.1 Назначение разных весов классам (cost-sensitive learning)
### 3.1.1 Модель XGBoost

In [28]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)

], remainder='passthrough')

In [29]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 250),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'eta': trial.suggest_float('eta', 0.01, 1.5)
    }

    estimator = xgb.XGBClassifier(**param, 
                                  scale_pos_weight=3.91,
                                  random_state=12345)
    
    xgb_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])

    score = cross_val_score(xgb_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [30]:
study_xgb_cw = optuna.create_study(study_name="XGBoost Optuna Optimization Class Weight",
                                direction='maximize')

In [31]:
study_xgb_cw.optimize(optuna_optimize, n_trials=100)

In [32]:
print("Наилучшие значения гиперпараметров XGBoost: {}".format(study_xgb_cw.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_xgb_cw.best_value))

Наилучшие значения гиперпараметров XGBoost: {'n_estimators': 80, 'max_depth': 2, 'eta': 0.2162809124337865}
f1_score на обучающем наборе: 0.609


In [33]:
with mlflow.start_run(nested=True,
                      run_name='xgb_pipeline_class_weight'):

    best_params = study_xgb_cw.best_params
    best_score = study_xgb_cw.best_value
    mlflow.log_params(best_params)

    xgb_pipe_cw = Pipeline([
        ('preprocessor', preprocessor),
        ('model', xgb.XGBClassifier(**best_params,
                                    scale_pos_weight=3.91,
                                    random_state=12345))
    ])

    xgb_pipe_cw.fit(X_train, y_train) 

    pred_proba = xgb_pipe_cw.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })

    mlflow.sklearn.log_model(xgb_pipe_cw, 'xgb_pipeline_class_weight')

### 3.1.2 Случайный лес

In [34]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300), # увеличить
        'max_depth': trial.suggest_int('max_depth', 2, 10), # увеличить
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 8)
    }

    estimator = RandomForestClassifier(**param, 
                                       class_weight='balanced',
                                       random_state=12345)

    rf_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])

    score = cross_val_score(rf_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [35]:
study_rf = optuna.create_study(study_name="RandomForest Optuna Optimization Class Weight",
                                direction='maximize')

In [36]:
study_rf.optimize(optuna_optimize, n_trials=100)

In [37]:
print("Наилучшие значения гиперпараметров Random Forest: {}".format(study_rf.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_rf.best_value))

Наилучшие значения гиперпараметров Random Forest: {'n_estimators': 293, 'max_depth': 9, 'min_samples_leaf': 4}
f1_score на обучающем наборе: 0.619


In [38]:
with mlflow.start_run(nested=True,
                      run_name='rf_pipeline_class_weight'):

    best_params = study_rf.best_params
    best_score = study_rf.best_value
    mlflow.log_params(best_params)

    rf_pipe_cw = Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestClassifier(**best_params,
                                         class_weight='balanced',
                                         random_state=12345))
    ])

    rf_pipe_cw.fit(X_train, y_train)

    pred_proba = rf_pipe_cw.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })
    
    mlflow.sklearn.log_model(rf_pipe_cw, 'rf_pipeline_class_weight')

### 3.1.3 Логистическая регрессия

In [39]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer()),
    ('scal', StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)
], remainder='passthrough')

In [40]:
def optuna_optimize(trial):
    param = {
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'saga'])
    }
    
    if param['solver'] == 'lbfgs':
        param['penalty'] = 'l2' 

    if param['solver'] == 'saga':
        param['penalty'] = 'elasticnet' 
        param['C'] = trial.suggest_float('C', 0.01, 100) 
        param['l1_ratio'] = trial.suggest_float('l1_ratio', 0, 1)
        param['max_iter'] = 1000 
        
    estimator = LogisticRegression(**param,  
                                   class_weight='balanced',
                                   random_state=12345)

    log_reg_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])
    
    score = cross_val_score(log_reg_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [41]:
study_log_reg = optuna.create_study(study_name="LogisticRegression Optuna Optimization Class Weight",
                                direction='maximize')

In [42]:
study_log_reg.optimize(optuna_optimize, n_trials=100)

In [43]:
print("Наилучшие значения гиперпараметров Logistic Regression: {}".format(study_log_reg.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_log_reg.best_value))

Наилучшие значения гиперпараметров Logistic Regression: {'solver': 'saga', 'C': 0.08339594924991811, 'l1_ratio': 0.46947428485072024}
f1_score на обучающем наборе: 0.493


In [44]:
with mlflow.start_run(nested=True,
                      run_name='log_reg_pipeline_class_weight'):

    best_params = study_log_reg.best_params
    best_score = study_log_reg.best_value
    mlflow.log_params(best_params)

    log_reg_pipe_cw = Pipeline([
        ('preprocessor', preprocessor),
        ('model', LogisticRegression(**best_params,
                                     class_weight='balanced',
                                     random_state=12345))
    ])

    log_reg_pipe_cw.fit(X_train, y_train)

    pred_proba = log_reg_pipe_cw.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })
    
    mlflow.sklearn.log_model(log_reg_pipe_cw, 'log_reg_pipeline_class_weight')

### 3.1.4 Метод опорных векторов

In [45]:
def optuna_optimize(trial):

    param = {
        'C': trial.suggest_float('C', 0.01, 3)  # какие значения может принимать С 
    }
 
    estimator = SVC(**param, 
                    class_weight='balanced',
                    random_state=12345,
                    probability=True)
    
    svc_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', estimator)
    ])
    
    score = cross_val_score(svc_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [46]:
study_svc = optuna.create_study(study_name="SVC Optuna Optimization Class Weight",
                                    direction='maximize')

In [47]:
study_svc.optimize(optuna_optimize, n_trials=100)

In [48]:
print("Наилучшие значения гиперпараметров SVC: {}".format(study_svc.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_svc.best_value))

Наилучшие значения гиперпараметров SVC: {'C': 2.6692787153761595}
f1_score на обучающем наборе: 0.580


In [49]:
with mlflow.start_run(nested=True,
                      run_name='svc_pipeline_class_weight'):

    best_params = study_svc.best_params
    best_score = study_svc.best_value
    mlflow.log_params(best_params)

    svc_pipe_cw = Pipeline([
        ('preprocessor', preprocessor),
        ('model', SVC(**best_params,
                      class_weight='balanced',
                      random_state=12345,
                      probability=True))
    ])

    svc_pipe_cw.fit(X_train, y_train)

    pred_proba = svc_pipe_cw.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })
    
    mlflow.sklearn.log_model(svc_pipe_cw, 'svc_pipeline_class_weight')

## 3.2 Генерация синтетических данных (SMOTE)
### 3.2.1 Модель XGBoost

In [50]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)
], remainder='passthrough')

In [51]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 250),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'eta': trial.suggest_float('eta', 0.01, 1.5)
    }

    estimator = xgb.XGBClassifier(**param,
                                  random_state=12345)

    xgb_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(xgb_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [52]:
study_xgb_smote = optuna.create_study(study_name="XGBoost Optuna Optimization SMOTE",
                                direction='maximize')

In [53]:
study_xgb_smote.optimize(optuna_optimize, n_trials=100)

In [54]:
print("Наилучшие значения гиперпараметров XGBoost: {}".format(study_xgb_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_xgb_smote.best_value))

Наилучшие значения гиперпараметров XGBoost: {'n_estimators': 116, 'max_depth': 4, 'eta': 0.12633443096280939}
f1_score на обучающем наборе: 0.624


In [55]:
with mlflow.start_run(nested=True,
                      run_name='xgb_pipeline_smote'):

    best_params = study_xgb_smote.best_params
    best_score = study_xgb_smote.best_value
    mlflow.log_params(best_params)

    xgb_pipe_sm = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', xgb.XGBClassifier(**best_params,
                                    random_state=12345))
    ])

    xgb_pipe_sm.fit(X_train, y_train)

    pred_proba = xgb_pipe_sm.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })

    mlflow.sklearn.log_model(xgb_pipe_sm, 'xgb_pipeline_smote')

### 3.2.2 Случайный лес

In [56]:
def optuna_optimize(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 8)
    }

    estimator = RandomForestClassifier(**param,
                                       random_state=12345)

    rf_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(rf_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [57]:
study_rf_smote = optuna.create_study(study_name="RandomForest Optuna Optimization SMOTE",
                               direction='maximize')

In [58]:
study_rf_smote.optimize(optuna_optimize, n_trials=100)

In [59]:
print("Наилучшие значения гиперпараметров RandomForest: {}".format(study_rf_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_rf_smote.best_value))

Наилучшие значения гиперпараметров RandomForest: {'n_estimators': 172, 'max_depth': 9, 'min_samples_leaf': 6}
f1_score на обучающем наборе: 0.615


In [60]:
with mlflow.start_run(nested=True,
                      run_name='rf_pipeline_smote'):

    best_params = study_rf_smote.best_params
    best_score = study_rf_smote.best_value
    mlflow.log_params(best_params)

    rf_pipe_sm = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', RandomForestClassifier(**best_params,
                                         random_state=12345))
    ])

    rf_pipe_sm.fit(X_train, y_train)

    pred_proba = rf_pipe_sm.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })
    
    mlflow.sklearn.log_model(rf_pipe_sm, 'rf_pipeline_smote')

### 3.2.3 Логистическая регрессия

In [61]:
cat_pipeline = Pipeline([
    ('enc', OneHotEncoder(handle_unknown='error', drop='first'))
])

num_pipeline = Pipeline([
    ('imp', SimpleImputer(fill_value=-1, add_indicator=True)),
    ('pt', PowerTransformer()),
    ('scal', StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", cat_pipeline, categorical_features),
    ("num", num_pipeline, numeric_features)
], remainder='passthrough')

In [62]:
def optuna_optimize(trial):
    param = {
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'saga'])
    }

    if param['solver'] == 'lbfgs':
        param['penalty'] = 'l2' 

    if param['solver'] == 'saga':
        param['penalty'] = 'elasticnet' 
        param['C'] = trial.suggest_float('C', 0.01, 100) # какие тут значения?
        param['l1_ratio'] = trial.suggest_float('l1_ratio', 0, 1)
        param['max_iter'] = 1000 

    estimator = LogisticRegression(**param,
                                   random_state=12345)

    log_reg_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(log_reg_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    return score

In [63]:
study_log_reg_smote = optuna.create_study(study_name="LogisticRegression Optuna Optimization SMOTE",
                                    direction='maximize')

In [64]:
study_log_reg_smote.optimize(optuna_optimize, n_trials=100)

In [65]:
print("Наилучшие значения гиперпараметров Logistic Regression: {}".format(study_log_reg_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_log_reg_smote.best_value))

Наилучшие значения гиперпараметров Logistic Regression: {'solver': 'saga', 'C': 0.010490817765909488, 'l1_ratio': 0.9943340963962667}
f1_score на обучающем наборе: 0.496


In [66]:
with mlflow.start_run(nested=True,
                      run_name='log_reg_pipeline_smote'):

    best_params = study_log_reg_smote.best_params
    best_score = study_log_reg_smote.best_value
    mlflow.log_params(best_params)

    log_reg_pipe_sm = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', LogisticRegression(**best_params,
                                     random_state=12345))
    ])

    log_reg_pipe_sm.fit(X_train, y_train)

    pred_proba = log_reg_pipe_sm.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })

    mlflow.sklearn.log_model(log_reg_pipe_sm, 'log_reg_pipeline_smote')

### 3.2.4 Метод опорных векторов

In [70]:
def optuna_optimize(trial):

    param = {
        'C': trial.suggest_float('C', 0.01, 3)  
    }

    estimator = SVC(**param,
                    random_state=12345,
                    probability=True)

    svc_pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', estimator)
    ])

    score = cross_val_score(svc_pipe, X_train, y_train, cv=5, scoring='f1').mean()
    
    return score

In [71]:
study_svc_smote = optuna.create_study(study_name="SVC Optuna Optimization SMOTE",
                                direction='maximize')

In [72]:
study_svc_smote.optimize(optuna_optimize, n_trials=10)

In [73]:
print("Наилучшие значения гиперпараметров SVC: {}".format(study_svc_smote.best_params))
print("f1_score на обучающем наборе: {:.3f}". format(study_svc_smote.best_value))

Наилучшие значения гиперпараметров SVC: {'C': 1.4864904056157744}
f1_score на обучающем наборе: 0.577


In [74]:
with mlflow.start_run(nested=True,
                      run_name='svc_pipeline_smote'):

    best_params = study_svc_smote.best_params
    best_score = study_svc_smote.best_value
    mlflow.log_params(best_params)

    svc_pipe_sm = Pipeline([
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=12345)),
        ('model', SVC(**best_params,
                      random_state=12345,
                      probability=True))
    ])

    svc_pipe_sm.fit(X_train, y_train)
    pred_proba = svc_pipe_sm.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    mlflow.log_metrics({
        'f1_cv': best_score,
        'roc_auc': roc_auc,
        'balanced_accuracy': balanced_accuracy_score(y_test, np.round(pred_proba, 0)),
        'f1': f1_score(y_test, np.round(pred_proba, 0))
    })

    mlflow.sklearn.log_model(svc_pipe_sm, 'svc_pipeline_smote')

In [75]:
colNames = ["XGBoost", "Random Forest", "Logistic Regression", "SVC"]
results = {"Imbalanced":[study_xgb.best_value, '-', '-', '-'],
           "Class Weight":[study_xgb_cw.best_value, 
                           study_rf.best_value, 
                           study_log_reg.best_value, 
                           study_svc.best_value],
           "SMOTE":[study_xgb_smote.best_value, 
                    study_rf_smote.best_value, 
                    study_log_reg_smote.best_value, 
                    study_svc_smote.best_value]}

In [76]:
# результаты метрики f1 score на кросс-валидации:
table = pd.DataFrame.from_dict(results, orient='index', columns=colNames)

In [77]:
table

,XGBoost,Random Forest,Logistic Regression,SVC
Imbalanced,0.590451,-,-,-
Class Weight,0.609432,0.619493,0.492849,0.579551
SMOTE,0.624178,0.615295,0.49579,0.576951


# 4. Тестирование лучшей модели


In [101]:
best_model = mlflow.sklearn.load_model('runs:/980b9744d00643c88633da47d8a2de6c/xgb_pipeline_smote')

In [102]:
joblib.dump(best_model, 'best_model.pkl')

['best_model.pkl']

In [103]:
pred_proba_xgb_smote = best_model.predict_proba(X_test)[:, 1]
print('ROC-AUC score на тестовой выборке:', roc_auc_score(y_test, pred_proba_xgb_smote))
print('f1 score на тестовой выборке:', f1_score(y_test, np.round(pred_proba_xgb_smote)))
print('Balanced accuracy на тестовой выборке:', balanced_accuracy_score(y_test, np.round(pred_proba_xgb_smote)))

ROC-AUC score на тестовой выборке: 0.8667357650408497
f1 score на тестовой выборке: 0.6383526383526384
Balanced accuracy на тестовой выборке: 0.7663757748503511


In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, pred_proba)

plt.figure(figsize=(6, 4))

plt.plot(fpr, tpr, label=f'XGBClassifier: {round(roc_auc_score(y_test, pred_proba), 4)}')

plt.plot([0, 1], [0, 1], linestyle='--', label='coin')

plt.xlim([-0.02, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.title('ROC-кривая')

plt.legend()

plt.show()